CAPSTONE TWO.

# 4. Pre-processing and Training Data Development

## 4.1 Introduction

In this chapter we continue working on creating ML model to predict whether a borrower will pay off a loan using the historical LendingClub data.

On our previous steps we alredy created the clean dataset: 

 * drop features with mising values
 * removed features with low variance
 * drop highly correlated features
 * convert categorical features to numeric
 * drop categorical features with many levels and constants
 
 Afre data cleaning and wrangling we reduced the number of our features from initial 151 to 60
 
 
 Then we used Recursive feature elimination algorithm to select optimal features.
 

## 4.2 Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from scipy import stats
# random enables us to generate random numbers
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold



## 4.3 Load the data

In [2]:
acc = pd.read_csv('accept_3_2.csv')

#### Features before we use Recursive features elimination :

In [3]:
acc.shape

(1023608, 60)

In [4]:
acc.columns

Index(['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'verification_status', 'loan_status', 'purpose', 'addr_state',
       'debt_settlement_flag', 'loan_amnt', 'int_rate', 'annual_inc', 'dti',
       'delinq_2yrs', 'fico_range_low', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_pymnt',
       'total_rec_int', 'total_rec_late_fee', 'recoveries', 'last_pymnt_amnt',
       'last_fico_range_high', 'last_fico_range_low', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'delinq_amnt',
       'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
       'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc',
       'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
       'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_accts', 'num_tl_90g_dpd_24m',
       'num_tl_o

## 4.4. Features Elimination 
(see CAPSTONE TWO. 3. Exploratory Data Analysis (EDA))

We used Recursive feature elimination algorithm to select optimal features.

To apply Recursive feature elimination algorithm with Cross-Validation 


    * we created dummy features for object type features 
    * standardized the magnitude of numeric features using a scaler
    

We saved the result datasets in .csv file

### Result fo the first 1,000 records - 70 optimal features:

In [5]:
X = pd.read_csv('accept_3_x.csv')

In [6]:
X.shape

(1000, 70)

In [7]:
X.columns

Index(['term', 'emp_length', 'loan_amnt', 'int_rate', 'annual_inc', 'dti',
       'delinq_2yrs', 'fico_range_low', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_pymnt',
       'total_rec_int', 'total_rec_late_fee', 'recoveries', 'last_pymnt_amnt',
       'last_fico_range_high', 'last_fico_range_low', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'total_bc_limit', 'grade_A',
       'grade_B', 'grade_D', 'grade_E', 'sub_grade_B5', 'sub_grade_D3',
  

#### target "loan status"  values (Fully paid = 1, Charged off = 0)

In [8]:
target =  pd.read_csv('accept_3_target.csv')

### Result for the 1,000 randomly seleced records  - following 9 optimal features:

In [9]:
XR = pd.read_csv('accept_3_xr.csv')

In [10]:
XR.shape

(1000, 9)

In [11]:
XR.head()

,loan_amnt,int_rate,total_pymnt,total_rec_int,recoveries,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,debt_settlement_flag_N
0,7400.0,19.99,7469.850167,69.85,0.00,7486.29,704.0,700.0,1
1,18250.0,11.14,20012.516107,1762.52,0.00,13426.96,724.0,720.0,1
2,28800.0,27.34,8892.990000,1865.21,6324.64,885.28,584.0,580.0,1
3,6200.0,5.93,6781.057552,581.06,0.00,188.40,659.0,655.0,1
4,6500.0,15.31,6182.160000,1498.68,214.84,535.64,514.0,510.0,1


#### targetR  "loan status"  values (Fully paid = 1, Charged off = 0)

In [12]:
targetR =  pd.read_csv('accept_3_target_r.csv')

## 4.5 Split into testing and training datasets 


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25)


In [14]:
XR_train, XR_test, yr_train, yr_test = train_test_split(XR, targetR, test_size=0.25)
